In [1]:
pip install kafka-python


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from kafka import KafkaProducer
from time import sleep
from json import dumps
import json

In [3]:

producer = KafkaProducer(bootstrap_servers=['54.224.245.218:9092'], #change ip here
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [18]:
producer.send('demo_test', value="{'nithish':'bethur'}")

In [8]:
df = pd.read_csv('/Users/nithishbilasunurmanjunathareddy/Documents/Stock Market project/indexProcessed.csv')

In [9]:
df.head()

,Index,Date,Open,High,Low,Close,Adj Close,Volume,CloseUSD
0,HSI,1986-12-31,2568.300049,2568.300049,2568.300049,2568.300049,2568.300049,0.0,333.879006
1,HSI,1987-01-02,2540.100098,2540.100098,2540.100098,2540.100098,2540.100098,0.0,330.213013
2,HSI,1987-01-05,2552.399902,2552.399902,2552.399902,2552.399902,2552.399902,0.0,331.811987
3,HSI,1987-01-06,2583.899902,2583.899902,2583.899902,2583.899902,2583.899902,0.0,335.906987
4,HSI,1987-01-07,2607.100098,2607.100098,2607.100098,2607.100098,2607.100098,0.0,338.923013


In [20]:
while True:
    dict_stock=df.sample(1).to_dict(orient="records")[0]
    producer.send('demo_test', value=dict_stock)
    sleep(1)

KeyboardInterrupt: 

In [23]:
import requests
import json
import time
from kafka import KafkaProducer

# API Key and Endpoint
apiKey = input("API Key: ")
apiUrl = "https://aeroapi.flightaware.com/aeroapi/"
airport = 'KSFO'
auth_header = {'x-apikey': apiKey}



API Key:  QcRJK8KcndprpZUL5Sqbyf2BYGvmdu0H


In [ ]:
# Kafka Setup
producer = KafkaProducer(
    bootstrap_servers='54.224.245.218:9092',  # Update with your Kafka server address
    value_serializer=lambda x: json.dumps(x).encode('utf-8')
)
kafka_topic = 'flight-tracking'  # Kafka topic for flight tracking data

def fetch_flight_data():
    payload = {'max_pages': 2}
    response = requests.get(apiUrl + f"airports/{airport}/flights",
                            params=payload, headers=auth_header)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error executing request")
        return None

def stream_to_kafka():
    while True:
        flight_data = fetch_flight_data()
        if flight_data:
            for flight in flight_data.get('flights', []):  # Adjust key based on API response
                producer.send(kafka_topic, flight)
                print(f"Sent to Kafka: {flight}")
        else:
            print("No data fetched")
        time.sleep(60)  # Fetch every 60 seconds

if __name__ == "__main__":
    stream_to_kafka()
